In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [13]:
columns_names=["user_id", "item_id", "rating", "timestamp"]
df=pd.read_csv("u.data", sep="\t", names=columns_names)
# We use 'sep' here because u.data is a tsv(tab sep value) file
movies_titles=pd.read_csv("u.item",sep="\|", header=None, engine='python')
# Here i'm providig engine='python' because 'c' engine does not support regex separators  
# print(movies_titles)
movies_titles=movies_titles[[0,1]]

movies_titles.columns=['item_id','title']
#print(movies_titles.head())
df= pd.merge(df, movies_titles, on="item_id")
# Merge the movies_titles dataframe with df on the item_id
#print(df)

In [4]:
sns.set_style('white')

In [12]:
df2=df.groupby('title').mean()['rating'].sort_values(ascending=False)
# Group by title and take the mean of rating and then sort these values in Descending order
#print(df2)
# So there may be a movie which rated by only one user and let say rated value is 5 so mean is also 5 which is not fair
# Now we take how many times a movie rated
df3=df.groupby('title').count()['rating'].sort_values(ascending=False)
# made a dataFrame of rating
ratings=pd.DataFrame(df.groupby('title').mean()['rating'])
# Add a column nums_of_ratings 
ratings['nums_of_ratings']=pd.DataFrame(df.groupby('title').count()['rating'])
ratings=ratings.sort_values(by='rating', ascending=False)
#plt.figure(figsize=(10,6))
#plt.hist(ratings['rating'], bins=70)
#plt.show()
# Draw a joint plot of ratings and num_of_ratings
#sn1=sns.jointplot(x='rating', y='nums_of_ratings', data=ratings, alpha=0.5)

moviemat=df.pivot_table(index="user_id", columns="title", values="rating")
#print(moviemat)
# Movie Matrix basically consist of user id and movie ratings
ratings=ratings.sort_values('nums_of_ratings',ascending=False)
# starwar_user_ratings=moviemat['Star Wars (1977)']
# #print(starwar_user_ratings)
# similar_to_startwars=moviemat.corrwith(starwar_user_ratings)
# corr_StarWars=pd.DataFrame(similar_to_startwars, columns=['Correlation'])
# #print(corr_StarWars)
# #Drop the NaN values
# corr_StarWars.dropna(inplace=True)
# corr_StarWars=corr_StarWars.sort_values('Correlation', ascending=False)
# #print(corr_StarWars)
# corr_StarWars=corr_StarWars.join(ratings['nums_of_ratings'])
# corr_StarWars=corr_StarWars[corr_StarWars['nums_of_ratings'] > 100].sort_values('Correlation', ascending=False)
#print(corr_StarWars)
def predict_movies(movie_name):
    movie_user_ratings=moviemat[movie_name]
    similar_to_movie=moviemat.corrwith(movie_user_ratings)
    corr_movie = pd.DataFrame(similar_to_movie, columns=['Correlation'])
    corr_movie.dropna(inplace=True)
    #corr_movie = corr_movie.sort_values('Correlation', ascending=False)
    corr_movie = corr_movie.join(ratings['nums_of_ratings'])
    predictions=corr_movie[corr_movie['nums_of_ratings'] > 100].sort_values('Correlation', ascending=False)
    return predictions
prediction=predict_movies("River Wild, The (1994)")
print(prediction.head())

                                 Correlation  nums_of_ratings
title                                                        
River Wild, The (1994)              1.000000              146
Kiss the Girls (1997)               0.744500              143
Young Guns (1988)                   0.662424              101
Seven Years in Tibet (1997)         0.640866              155
My Best Friend's Wedding (1997)     0.576746              172


# End of the project